In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import scipy.stats as stats
import xgboost as xgb
import json
import gc
import os
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from google.colab import drive
drive.mount('/content/drive')

test = pl.read_parquet('/content/drive/MyDrive/Colab Notebooks/JS_kaggle/Model/data/train.parquet/partition_id=9/SAMPLE.parquet')

display(test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1616,306,31,1.239067,1.459457,0.493426,1.7203,1.226069,-0.059208,0.561246,0.372741,0.724132,-0.995373,44,3,24,-0.525398,-0.008198,-0.440766,-0.460561,-0.469248,-0.481003,0.415355,-0.184878,-0.112169,0.033508,-0.689095,-0.261817,0.389375,-0.942774,0.004519,-2.040649,-2.398883,-0.635655,-0.929971,0.015156,0.25239,…,-0.610722,1.320908,1.307829,-0.412767,0.003096,0.907029,-0.453041,0.850575,0.571426,2.290587,0.996665,-0.38572,-0.177393,-0.228848,0.063149,-0.260378,-0.314975,-0.057074,-0.489596,-0.508503,0.065038,-0.243509,-0.194948,-0.164707,-0.232259,-0.206016,-0.17737,-0.12822,-0.055265,-0.176105,-0.00829,-0.330336,-0.373021,-0.533409,-0.474922,-0.172897,-0.771277
1532,483,0,2.985356,0.090435,-0.082673,0.15249,0.412641,-0.694304,-0.613056,-0.035183,-0.801999,-0.738475,11,7,76,-0.696615,-0.328094,-0.307807,-0.852785,-0.776617,-0.683479,-0.875004,0.899584,-0.008123,-0.110043,1.423861,0.848555,1.420545,0.557609,0.901168,0.965627,0.190484,-0.85688,-1.023805,-0.124994,-0.160664,…,-0.804731,-1.173743,0.568221,0.702137,0.768969,0.634936,1.128178,-0.252717,-0.051346,-0.218409,1.024469,-0.504533,-0.326237,-0.36117,-0.815238,0.813816,-0.730725,-0.340653,-0.596432,-0.47599,-0.299513,-0.622661,-0.126248,-0.066152,-0.079563,-0.038699,0.011575,0.076288,-0.09499,0.023367,-0.133566,-0.005384,-0.012246,0.080417,0.046046,-0.019267,0.194889
1660,740,23,2.031088,1.442209,-0.283681,1.047328,1.547427,1.073373,0.978737,0.101395,0.746582,-0.70562,34,4,214,-0.691657,0.389847,0.037869,-0.46571,-0.350288,-0.528171,0.026385,1.116866,0.140697,-0.281586,0.169476,0.1309,0.275927,-0.335944,2.068369,1.289827,0.434819,-0.538434,-0.398074,-0.273089,1.47073,…,1.172604,0.619513,1.352012,-0.820089,0.501652,0.577048,1.810872,0.516981,0.025291,1.146231,-0.590175,-0.262729,-0.201081,-0.429892,0.040754,1.570194,-0.852148,0.242955,-0.262128,-0.201149,0.376248,0.380029,-0.301237,-0.306333,0.280676,0.431549,-0.158481,-0.160151,-1.213278,-2.248488,-0.251419,0.161664,-0.589093,-0.125467,0.752429,0.329659,-0.020278
1577,194,25,3.699438,2.159096,1.873386,2.175647,2.181123,0.826584,-0.401185,-0.297226,-0.11992,0.396108,57,1,230,0.53922,-0.176115,0.348248,-0.547107,-0.69354,-0.939209,-1.825719,-0.346283,-0.061464,0.39436,1.382024,0.830699,3.34555,2.201022,-0.604593,-0.375263,-0.445077,-0.746108,-0.854946,0.405621,0.840731,…,-0.770201,0.417427,-0.845799,-0.298539,1.000279,-0.843617,-1.262755,-0.710786,0.017076,-0.193152,-0.064444,-0.37764,-0.288108,-0.253723,-2.2879,0.014053,0.620506,-0.142283,0.324024,0.816142,-0.171743,0.13854,0.087688,0.004184,-0.075801,-0.234022,0.057866,-0.021643,-0.316436,-0.028188,-0.141699,-0.365251,0.588478,-0.009456,-0.454092,0.746538,0.101324
1639,759,31,1.305348,-1.01555,0.594645,-0.361093,-0.823819,1.036514,0.533917,-0.223178,-0.488847,0.023087,44,3,24,0.264393,-0.304043,-0.267793,-0.464585,-0.487466,-0.778418,1.544372,0.804458,0.489448,-0.081229,-0.681175,-0.365

In [ ]:
#Change column type so we do not end up overflowing
test = test.with_columns([
    pl.col('time_id').cast(pl.Int32),
    pl.col('date_id').cast(pl.Int32)
])


#Combining date and time columns into a combined_timestamp representing minutes (1440 mins in a day)
test = test.with_columns(
    (pl.col('date_id') * 1440 + pl.col('time_id')).alias('combined_timestamp')
    )


#Add weekly and daily cycles
test = test.with_columns([
    (2 * np.pi * pl.col("time_id") / 1440).sin().alias("daily_cycle_sin"),
    (2 * np.pi * pl.col("time_id") / 1440).cos().alias("daily_cycle_cos")
])


test = test.with_columns([
    (2 * np.pi * pl.col("date_id") / 1440).sin().alias("weekly_cycle_sin"),
    (2 * np.pi * pl.col("date_id") / 1440).cos().alias("weekly_cycle_cos")
])
weightsSettest = test.select(['weight'])
responderTest = test.select(['responder_6'])

#Add encoding to symbol_id
symbol_encoding = (
    test.select("symbol_id")
    .unique()
    .with_row_index(name="symbol_id_encoded")
)

test = test.join(symbol_encoding, on="symbol_id")

columns_to_drop = ['weight', 'date_id', 'time_id', 'symbol_id']
columns_to_drop += [f"responder_{i}" for i in range(9)]
test = test.drop(columns_to_drop)

gc.collect()

display(test)
display(weightsSettest)

feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,combined_timestamp,daily_cycle_sin,daily_cycle_cos,weekly_cycle_sin,weekly_cycle_cos,symbol_id_encoded
f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,f64,f64,f64,f64,u32
1.459457,0.493426,1.7203,1.226069,-0.059208,0.561246,0.372741,0.724132,-0.995373,44,3,24,-0.525398,-0.008198,-0.440766,-0.460561,-0.469248,-0.481003,0.415355,-0.184878,-0.112169,0.033508,-0.689095,-0.261817,0.389375,-0.942774,0.004519,-2.040649,-2.398883,-0.635655,-0.929971,0.015156,0.25239,1.101797,-0.178528,0.768788,-0.631361,…,-0.074464,1.646446,1.794934,-0.610722,1.320908,1.307829,-0.412767,0.003096,0.907029,-0.453041,0.850575,0.571426,2.290587,0.996665,-0.38572,-0.177393,-0.228848,0.063149,-0.260378,-0.314975,-0.057074,-0.489596,-0.508503,0.065038,-0.243509,-0.194948,-0.164707,-0.232259,-0.206016,-0.17737,-0.12822,2327346,0.97237,0.233445,0.694658,0.71934,23
0.090435,-0.082673,0.15249,0.412641,-0.694304,-0.613056,-0.035183,-0.801999,-0.738475,11,7,76,-0.696615,-0.328094,-0.307807,-0.852785,-0.776617,-0.683479,-0.875004,0.899584,-0.008123,-0.110043,1.423861,0.848555,1.420545,0.557609,0.901168,0.965627,0.190484,-0.85688,-1.023805,-0.124994,-0.160664,0.178538,-0.412069,-0.186708,-0.309237,…,-0.028578,-0.246345,-2.230528,-0.804731,-1.173743,0.568221,0.702137,0.768969,0.634936,1.128178,-0.252717,-0.051346,-0.218409,1.024469,-0.504533,-0.326237,-0.36117,-0.815238,0.813816,-0.730725,-0.340653,-0.596432,-0.47599,-0.299513,-0.622661,-0.126248,-0.066152,-0.079563,-0.038699,0.011575,0.076288,2206563,0.859406,-0.511293,0.390731,0.920505,10
1.442209,-0.283681,1.047328,1.547427,1.073373,0.978737,0.101395,0.746582,-0.70562,34,4,214,-0.691657,0.389847,0.037869,-0.46571,-0.350288,-0.528171,0.026385,1.116866,0.140697,-0.281586,0.169476,0.1309,0.275927,-0.335944,2.068369,1.289827,0.434819,-0.538434,-0.398074,-0.273089,1.47073,-0.372058,1.587579,1.950976,-0.201971,…,-0.215488,0.625374,1.005136,1.172604,0.619513,1.352012,-0.820089,0.501652,0.577048,1.810872,0.516981,0.025291,1.146231,-0.590175,-0.262729,-0.201081,-0.429892,0.040754,1.570194,-0.852148,0.242955,-0.262128,-0.201149,0.376248,0.380029,-0.301237,-0.306333,0.280676,0.431549,-0.158481,-0.160151,2391140,-0.087156,-0.996195,0.819152,0.573576,30
2.159096,1.873386,2.175647,2.181123,0.826584,-0.401185,-0.297226,-0.11992,0.396108,57,1,230,0.53922,-0.176115,0.348248,-0.547107,-0.69354,-0.939209,-1.825719,-0.346283,-0.061464,0.39436,1.382024,0.830699,3.34555,2.201022,-0.604593,-0.375263,-0.445077,-0.746108,-0.854946,0.405621,0.840731,1.38843,1.492564,1.713243,-0.365286,…,-0.06645,-0.318986,-0.222446,-0.770201,0.417427,-0.845799,-0.298539,1.000279,-0.843617,-1.262755,-0.710786,0.017076,-0.193152,-0.064444,-0.37764,-0.288108,-0.253723,-2.2879,0.014053,0.620506,-0.142283,0.324024,0.816142,-0.171743,0.13854,0.087688,0.004184,-0.075801,-0.234022,0.057866,-0.021643,2271074,0.748956,0.66262,0.562805,0.82659,21
-1.01555,0.594645,-0.361093,-0.823819,1.036514,0.533917,-0.223178,-0.488847,0.023087,44,3,24,0.264393,-0.304043,-0.267793,-0.464585,-0.487466,-0.778418,1.544372,0.8

weight
f32
1.239067
2.985356
2.031088
3.699438
1.305348
…
2.939481
2.358879
1.067876


In [ ]:
X_test = test.to_pandas()
weightsSettest = weightsSettest.to_pandas()
responderTest = responderTest.to_pandas()
gc.collect()

display(responderTest)
display(X_test)
display(weightsSettest)

,responder_6
0,-0.474922
1,0.046046
2,0.752429
3,-0.454092
4,0.283637
...,...
627452,-0.421453
627453,0.592278
627454,0.085982
627455,0.122812


,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,...,feature_75,feature_76,feature_77,feature_78,combined_timestamp,daily_cycle_sin,daily_cycle_cos,weekly_cycle_sin,weekly_cycle_cos,symbol_id_encoded
0,1.459457,0.493426,1.720300,1.226069,-0.059208,0.561246,0.372741,0.724132,-0.995373,44,...,-0.232259,-0.206016,-0.177370,-0.128220,2327346,0.972370,0.233445,0.694658,0.719340,23
1,0.090435,-0.082673,0.152490,0.412641,-0.694304,-0.613056,-0.035183,-0.801999,-0.738475,11,...,-0.079563,-0.038699,0.011575,0.076288,2206563,0.859406,-0.511293,0.390731,0.920505,10
2,1.442209,-0.283681,1.047328,1.547427,1.073373,0.978737,0.101395,0.746582,-0.705620,34,...,0.280676,0.431549,-0.158481,-0.160151,2391140,-0.087156,-0.996195,0.819152,0.573576,30
3,2.159096,1.873386,2.175647,2.181123,0.826584,-0.401185,-0.297226,-0.119920,0.396108,57,...,-0.075801,-0.234022,0.057866,-0.021643,2271074,0.748956,0.662620,0.562805,0.826590,21
4,-1.015550,0.594645,-0.361093,-0.823819,1.036514,0.533917,-0.223178,-0.488847,0.023087,44,...,-0.270771,-0.435213,-0.186153,-0.292942,2360919,-0.169350,-0.985556,0.763232,0.646124,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627452,0.822967,0.499626,0.944325,1.194925,-1.453991,0.458592,0.320457,0.826869,1.004735,15,...,-0.224471,-0.334782,-0.154452,-0.291640,2344726,0.979925,-0.199368,0.731354,0.681998,1
627453,3.451627,2.195243,3.332921,2.749668,1.897658,-1.037757,-0.290979,-0.049235,0.643994,82,...,-0.174155,-0.127680,-0.035999,0.010003,2436530,0.216440,0.976296,0.891007,0.453990,2
627454,2.857555,-0.637666,2.948770,2.900311,1.104987,0.934380,0.969601,1.753939,0.127401,11,...,-0.326492,-0.241095,-0.269414,-0.260370,2222727,-0.370557,-0.928810,0.434445,0.900698,38
627455,1.999116,-0.307108,1.929869,2.209296,-1.165422,-0.884001,-0.064150,-0.706430,-1.255206,82,...,-2.340077,-2.994808,-2.178871,-2.568394,2323475,-0.152123,-0.988362,0.685183,0.728371,2


,weight
0,1.239067
1,2.985356
2,2.031088
3,3.699438
4,1.305348
...,...
627452,2.939481
627453,2.358879
627454,1.067876
627455,2.304580


In [ ]:
def pathGeneration(part):
  return f'/content/drive/MyDrive/Colab Notebooks/JS_kaggle/Model/Meta models/ds{part}'

class Cluster:
  def __init__(self, part):
    self.path = pathGeneration(part)
    self.models = []
    self.load_models()

  def load_model(self, responder):
    file_path = os.path.join(self.path, f'responder_0{responder}.json')
    model = xgb.XGBRegressor()
    model.load_model(file_path)
    return model

  def load_models(self):
    for responder in range(9):
      model = self.load_model(responder)
      self.models.append(model)

  def predict(self, testSet, responder):
    model = self.models[responder]
    return model.predict(testSet)

  def predict_all(self, testSet):
    predictions = []
    for responder in range(9):
      prediction = self.predict(testSet, responder)
      prediction = pd.DataFrame(prediction)
      predictions.append(prediction)
    return predictions

In [ ]:
#Load all models
meta = []
for i in range(10):
  meta.append(Cluster(i))

In [ ]:
#Load all predictions
metaPredictions = []
for i in range(10):
  metaPredictions.append(meta[i].predict_all(X_test))

In [ ]:
#Organize predictions
flatPredictions = [df.squeeze() for dfList in metaPredictions for df in dfList]
combinePredictions = pd.concat(flatPredictions, axis=1)

In [ ]:
#Set column names
combinePredictions.columns = [f"cluster_{j}_responder_{i}" for j in range(10) for i in range(9)]
display(combinePredictions)

,cluster_0_responder_0,cluster_0_responder_1,cluster_0_responder_2,cluster_0_responder_3,cluster_0_responder_4,cluster_0_responder_5,cluster_0_responder_6,cluster_0_responder_7,cluster_0_responder_8,cluster_1_responder_0,...,cluster_8_responder_8,cluster_9_responder_0,cluster_9_responder_1,cluster_9_responder_2,cluster_9_responder_3,cluster_9_responder_4,cluster_9_responder_5,cluster_9_responder_6,cluster_9_responder_7,cluster_9_responder_8
0,-2.417108,1.119835,-0.717367,-0.916069,0.394479,-1.863493,0.185159,0.333724,0.149757,-0.945401,...,0.177226,-0.263681,-0.086979,-0.452741,-0.061703,0.091523,-0.202847,0.045281,0.166778,0.072485
1,-0.162221,0.332873,-0.147420,0.071552,0.237519,0.544139,0.113264,0.360485,0.061918,-0.100820,...,0.021573,-0.034921,-0.022399,-0.048119,0.050707,0.333901,-0.003555,0.069372,0.315793,0.030206
2,-0.487285,1.063126,-0.113766,-0.092798,0.568085,-0.258755,0.145130,0.498386,0.072362,-0.301503,...,-0.004416,-0.145816,-0.238903,-0.212862,-0.079846,0.047317,-0.078124,-0.016394,0.103477,0.040094
3,-0.698861,-0.087564,0.241899,-0.108404,0.964070,-0.709522,-0.108903,0.392544,-0.082149,-0.357554,...,0.084305,-0.107457,-0.188649,-0.162147,0.022775,-0.069480,-0.026464,-0.030518,0.010041,0.065798
4,-1.078924,0.262343,-0.029117,-0.815989,0.228908,-0.326867,-0.685891,-1.105495,-0.536295,-0.392656,...,0.051624,0.125535,-0.012446,0.201782,-0.051295,0.175128,0.077735,-0.082616,0.024634,-0.090610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627452,-1.580026,-0.595674,0.296941,-0.906905,-0.637124,-0.941134,0.192194,0.396747,-0.504980,0.109048,...,-0.091562,0.055546,-0.066445,0.081347,-0.023244,0.011180,-0.009936,-0.073916,-0.067186,-0.069935
627453,-0.075112,0.217416,0.617760,0.616775,1.575419,-0.330326,2.076107,1.684469,0.567728,0.090547,...,0.413926,0.043958,0.124113,-0.021003,0.313785,-0.049753,0.051738,0.121746,-0.026782,0.056147
627454,0.337118,0.753290,0.493429,0.244392,0.575002,-1.348611,0.345967,1.093578,-0.721285,0.228252,...,-0.303663,0.283041,0.212912,0.306495,-0.025419,-0.021026,-0.065419,-0.264205,-0.330555,-0.295986
627455,-0.552850,-0.258963,-0.167691,-0.036251,2.525903,-0.437812,0.462683,1.476536,-0.507786,-0.033181,...,-0.004448,-0.107937,-0.110649,-0.208148,-0.023713,0.072696,-0.038366,0.077516,0.314424,0.059733


In [ ]:
from sklearn.metrics import r2_score

r2_scores = {col: r2_score(responderTest, combinePredictions[col]) for col in combinePredictions.columns}
display(r2_scores)
raw_weights = np.array([max(0, r2) for r2 in r2_scores.values()])
normalized_weights = raw_weights / raw_weights.sum()
weights = dict(zip(combinePredictions.columns, normalized_weights))
weighted_avg = sum(weights[model] * combinePredictions[model] for model in combinePredictions.columns)
weighted_avg = pd.Series(weighted_avg, index=responderTest.index)
display(weighted_avg)
final_r2 = r2_score(responderTest, weighted_avg)
print(f"Weighted Average R^2: {final_r2}")

{'cluster_0_responder_0': -2.084320306777954,
 'cluster_0_responder_1': -1.5197019577026367,
 'cluster_0_responder_2': -0.7378641366958618,
 'cluster_0_responder_3': -0.9046355485916138,
 'cluster_0_responder_4': -1.4990692138671875,
 'cluster_0_responder_5': -0.7174906730651855,
 'cluster_0_responder_6': -0.36630773544311523,
 'cluster_0_responder_7': -0.7264645099639893,
 'cluster_0_responder_8': -0.40658366680145264,
 'cluster_1_responder_0': -0.6510964632034302,
 'cluster_1_responder_1': -1.0439813137054443,
 'cluster_1_responder_2': -0.505079984664917,
 'cluster_1_responder_3': -0.582482099533081,
 'cluster_1_responder_4': -1.4823009967803955,
 'cluster_1_responder_5': -0.24658429622650146,
 'cluster_1_responder_6': -0.3602616786956787,
 'cluster_1_responder_7': -1.117448091506958,
 'cluster_1_responder_8': -0.1860295534133911,
 'cluster_2_responder_0': -0.27139246463775635,
 'cluster_2_responder_1': -0.4610503911972046,
 'cluster_2_responder_2': -0.27112865447998047,
 'cluster_2_

,0
0,0.080896
1,0.118485
2,0.026012
3,0.002752
4,-0.059022
...,...
627452,-0.071337
627453,0.070279
627454,-0.287803
627455,0.129603


Weighted Average R^2: 0.17467916011810303


In [ ]:
!pip install pykalman
from pykalman import KalmanFilter

kf = KalmanFilter(initial_state_mean=0.080896, n_dim_obs=1)

state_means, _ = kf.smooth(weighted_avg.values.reshape(-1, 1))
smoothed_predictions = pd.Series(state_means.flatten(), index=responderTest.index)

smoothed_r2 = r2_score(responderTest, smoothed_predictions)
print(f"Smoothed Predictions R^2: {smoothed_r2}")

Smoothed Predictions R^2: 0.09230191416516409
